# Chapter 06 출력 파서

- 출력 파서는 언어 모델의 출력값을 더 유용하고 구조화된 형태로 해석해서 변환(파싱)하는 중요한 컴포넌트

- **출력 파서**는 LLM의 출력값을 구조화된 형식으로 변환하고 답변에서 원하는 정보만 뽑아낼 때 유용하게 사용되는 도구
- 수동으로 해석하고 필요한 정보를 추출하려면 너무 어렵다
- 출력 파서는 사전에 정의된 양식이 있으면 파이썬으로 원하는 키를 조회해서 그에 해당하는 값을 쉽게 추출 가능
- 출력 파서의 주요 특징
  - 다양성: 많은 종류의 출력 파서가 있어 다양한 변환 방식에 대응 가능
  - 스트리밍 지원: 실시간 처리 가능
  - 확장성: 인터페이스를 통해 최소한의 모듈에서 복잡한 모듈까지 확장 가능
- 출력 파서의 이점
  - 구조화
  - 일관성
  - 유연성

## 01 PydanticOuputParser

- **Pydantic**은 파이썬에서 데이터 유효성을 검사하고, 구조화된 형식으로 관리하는 데 유용한 라이브러리
- **유효성 검사**: 데이터가 정해진 조건이나 형식에 맞는지 확인하여 잘못된 값이 입력되지 않도록 하는 도구
- `PydanticOutputParser`는 언어 모델의 출력을 **구조화된 정보**로 변환하는 데 도움을 주는 클래스
- 이 클래스는 단순 텍스트 응답 대신 **명확하고 체계적인 형태로 필요한 정보를 제공**가능
- 이 클래스를 활용하면 언어 모델의 출력을 특정 데이터 모델에 맞게 변환하여 정보를 더 쉽게 처리하고 활용 가능
- `PydanticOutputParser` (대부분의 OutputParser에 해당)에는 주로 **두 가지 핵심 메서드**가 구현되어야 함
  - **`get_format_instructions()`**
     - 언어 모델이 출력해야 할 정보의 형식을 정의하는 지침을 제공
     - 예를 들어, 언어 모델이 출력해야 할 데이터의 필드와 그 형태를 설명하는 지침을 문자열로 반환 가능
     - 이 지침은 언어 모델이 출력을 구조화하고 특정 데이터 모델에 맞게 변환하는 데 매우 중요
  - **`parse()`**:
    - 언어 모델의 출력(문자열로 가정)을 받아 이를 특정 구조로 분석하고 변환 가능
    - pydantic과 같은 도구를 사용하여 입력된 문자열을 사전 정의된 스키마에 따라 검증하고, 해당 스키마를 따르는 데이터 구조로 변환

In [53]:
from dotenv import load_dotenv
from langchain_teddynote import logging
from langchain_teddynote.messages import stream_response
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

load_dotenv()
logging.langsmith("CH03-OutputParser")
llm = ChatOpenAI(temperature=0, model_name="gpt-4.1-mini")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH03-OutputParser


In [54]:
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

In [55]:
from itertools import chain
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    "다음의 이메일 내용중 중요한 내용을 추출해 주세요.\n\n{email_conversation}"
)

llm = ChatOpenAI(temperature=0, model_name="gpt-4.1-mini")

chain = prompt | llm

answer = chain.stream({"email_conversation": email_conversation})

output = stream_response(answer, return_output=True)

다음은 이메일의 중요한 내용입니다:

1. 발신자: 김철수 상무 (바이크코퍼레이션)
2. 수신자: 이은채 대리 (테디인터내셔널)
3. 요청 사항:
   - "ZENESIS" 자전거에 대한 상세 브로슈어 요청
   - 특히 기술 사양, 배터리 성능, 디자인 정보 필요
4. 목적: 유통 전략 및 마케팅 계획 구체화를 위한 정보 확보
5. 미팅 제안:
   - 일시: 1월 15일 화요일 오전 10시
   - 장소: 귀사 사무실
   - 내용: 협력 가능성 논의

- 위와 같은 이메일 내용이 주어졌을 때 아래의 Pydantic 스타일로 정의된 클래스를 사용하여 이메일의 정보를 파싱
- Field 안에 `description` 은 텍스트 형태의 답변에서 주요 정보를 추출하기 위한 설명
- LLM 이 바로 이 설명을 보고 필요한 정보를 추출. 그러므로 이 설명은 정확하고 명확해야 한다

In [56]:
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="보낸 사람의 이메일 주소")
    subject: str = Field(description="이메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")

In [57]:
parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [58]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일을 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "이메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일 본문에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}
```


- 프롬프트를 정의

1. `question`: 유저의 질문
2. `email_conversation`: 이메일 본문의 내용
3. `format`: 형식을 지정

In [59]:
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

- question과 email_conversation은 나중에 입력받으므로, 부분 변수로 format만 채운다

In [60]:
# format 에 PydanticOutputParser의 부분 포맷팅(partial) 추가
prompt = prompt.partial(format=parser.get_format_instructions())
prompt

PromptTemplate(input_variables=['email_conversation', 'question'], input_types={}, partial_variables={'format': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"person": {"description": "메일을 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "이메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일 본문에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": 

In [61]:
chain = prompt | llm
response = chain.stream(
    {
        "question": "이메일 내용 중 주요 내용을 추출해주세요",
        "email_conversation": email_conversation,
    }
)
output = stream_response(response, return_output=True)

```json
{
  "person": "김철수",
  "email": "chulsoo.kim@bikecorporation.me",
  "subject": "\"ZENESIS\" 자전거 유통 협력 및 미팅 일정 제안",
  "summary": "바이크코퍼레이션 김철수 상무가 ZENESIS 자전거의 상세 브로슈어(기술 사양, 배터리 성능, 디자인) 요청과 유통 및 마케팅 협력 가능성 논의를 위해 미팅을 제안함.",
  "date": "1월 15일 오전 10시"
}
```

- output 출력값을 보면 dictionary 형태지만 문자열임
- parser를 사용하여 결과를 파싱

In [62]:
structured_output = parser.parse(output)
print(structured_output)

# 객체니깐 필드에 쉽게 참조
structured_output.person

person='김철수' email='chulsoo.kim@bikecorporation.me' subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안' summary='바이크코퍼레이션 김철수 상무가 ZENESIS 자전거의 상세 브로슈어(기술 사양, 배터리 성능, 디자인) 요청과 유통 및 마케팅 협력 가능성 논의를 위해 미팅을 제안함.' date='1월 15일 오전 10시'


'김철수'

- 출력 파서 자체를 체인에 추가 가능
- 체인을 실행하면 결과값 자체가 객체로 변환

In [63]:
chain = prompt | llm | parser

# chain 을 실행하고 결과를 출력
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# 결과는 EmailSummary 객체 형태로 출력
response

EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary='바이크코퍼레이션 김철수 상무가 ZENESIS 자전거에 대한 상세 브로슈어(기술 사양, 배터리 성능, 디자인)를 요청하고, 유통 전략과 마케팅 계획을 구체화하기 위해 협력 가능성을 논의하고자 미팅을 제안함.', date='1월 15일 화요일 오전 10시')

## 02 with_structured_output

- `with_structured_output()`을 사용하여 출력 파서를 추가하면, 출력을 Pydantic 객체로 변환 가능

- 이 답변은 구조화된 답변 X

In [64]:
llm = ChatOpenAI(
    temperature=0, model_name="gpt-4o"
)
llm.invoke("대한민국의 수도는 뭐야?")

AIMessage(content='대한민국의 수도는 서울입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 15, 'total_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_ad98c18a04', 'id': 'chatcmpl-D5cRFtaincy5kHao2LcX6LjTbxVij', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--019c2a09-283e-7c20-b611-509f5c6c18ac-0', usage_metadata={'input_tokens': 15, 'output_tokens': 8, 'total_tokens': 23, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [65]:
llm_with_structered = ChatOpenAI(
    temperature=0, model_name="gpt-4o"
).with_structured_output(EmailSummary)

answer = llm_with_structered.invoke(email_conversation)
answer.person

'김철수'

- 한 가지 아쉬운 점은 `with_structured_output()` 함수는 `stream()` 기능을 지원 X

## 03 LangSmith에서 출력 파서의 흐름 파악하기

- RAG 시스템을 만들면서 구조화된 답변이 제대로 나오지 않는 경우가 생길 수 있음
- 이럴 때는 LangSmith 에서 출력 파서의 흐름을 확인하는 것이 좋음